In [2]:
%matplotlib inline
import os
os.chdir('U:\\GitHubClones\\SALTSeniorDesign')
os.getcwd()
#import Excel
import xlwings as xw
import seaborn as sb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import re
from datetime import datetime
import datetime
import statsmodels.formula.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn import preprocessing
import time

In [3]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
cols = ['assetNumber', 'reportingPeriodEndingDate','reportingPeriodBeginningDate',
        'originalLoanTerm','originalLoanAmount','vehicleValueAmount','originalInterestRatePercentage','subvented',
        'vehicleNewUsedCode', 'vehicleModelYear','vehicleTypeCode',
        'obligorCreditScore','obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode','obligorGeographicLocation',
        'paymentToIncomePercentage', 'reportingPeriodActualEndBalanceAmount',
        'reportingPeriodBeginningLoanBalanceAmount','totalActualAmountPaid',
        'remainingTermToMaturityNumber',
        'currentDelinquencyStatus','coObligorIndicator',
        'zeroBalanceCode']

logitRemoveCols = ['assetNumber', 'reportingPeriodEndingDate','originalLoanAmount','vehicleValueAmount','reportingPeriodActualEndBalanceAmount',
        'reportingPeriodBeginningLoanBalanceAmount','totalActualAmountPaid','vehicleNewUsedCode','zeroBalanceCode',
        'vehicleModelYear','currentDelinquencyStatus','subvented','obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode',
        'obligorGeographicLocation','reportingPeriodBeginningDate']

In [5]:
wb = xw.Book('Salt.xlsm')
setup = wb.sheets('Setup')
output = wb.sheets('Output')
strats = wb.sheets('Strat Tables')
performance = wb.sheets('Performance')

# Path For the Current deal that is selected based on Setup
directory = setup['Setup.Directory'].value
shelf = setup['Setup.Shelf'].value
shelfList = setup['Shelf_List'].value
deal = setup['Setup.Deal'].value
report = setup['Setup.Report'].value
num_Deals = setup['Setup.Num_Deals'].value
max_Report = setup['Setup.Max_Report_Num'].value


In [6]:
tapes = np.array(setup['Tapes'].value)
tapes = tapes[0:int(max_Report),0:int(num_Deals)]
dfTapes = pd.DataFrame(data=tapes[1:,1:], columns=tapes[0,1:])
dealList = list(dfTapes.columns)

In [10]:
def chargeOffs(trainOrTest, tapes, dealList, shelfList, unemploymentPath):
    shelfList = list(filter(lambda a: a != 'None', shelfList))
    train = pd.DataFrame()
    for k in range(3, len(dealList)):
        print(dealList[k])
        monthList = list(tapes[dealList[k]].astype(str))
        monthList = list(filter(lambda a: a != 'None', monthList))
        monthList = monthList[:-1]
        
        if trainOrTest == 'Test':
            monthList = [monthList[-1]]
        
        for i in range(0, len(monthList)):
            path = directory + '\\' + shelfList[k] + '\\' + dealList[k] + '\\' + monthList[i]
            df = pd.read_csv(path, usecols=cols, memory_map=True, low_memory=False)
            df = df[~df.zeroBalanceCode.isin(['-'])]#taking out all of the performing loans so far

            df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount'])*100
            df['loanDiff'] = (df['originalLoanAmount'] - df['reportingPeriodBeginningLoanBalanceAmount'])
            df['coObligorIndicator'] = df['coObligorIndicator'].astype(int)

            umrate = pd.read_csv(unemploymentPath, memory_map=True, low_memory=False)
            umrate['Date'] = pd.to_datetime(umrate.Date)
            umrate['Date'] = umrate['Date'].dt.strftime('%m/%d/%Y')
            umrate = umrate.set_index('Date')
            stateDict = umrate.to_dict(orient='index')

            df['reportingPeriodBeginningDate'] = pd.to_datetime(df['reportingPeriodBeginningDate']).dt.strftime('%m/%d/%Y')
            states = list(df['obligorGeographicLocation'])
            uniqueList = df.reportingPeriodBeginningDate.unique()
            umList = []
            for j in range(0, len(states)):
                if not uniqueList[0]:
                    continue
                val = stateDict[uniqueList[0]].get(states[j])
                umList.append(val)
            unemployment = pd.Series(umList)
            df = df.assign(StateUnemployment=unemployment.values)
            train = train.append(df)
    
    
    return train

In [8]:
def clean(data):
    data['obligorCreditScore'] = data['obligorCreditScore'].replace(['-'], 0)
    data['zeroBalanceCode'] = pd.to_numeric(data['zeroBalanceCode'], errors = 'coerce')
    data['obligorCreditScore'] = pd.to_numeric(data['obligorCreditScore'], errors = 'coerce')
    data['subvented'] = pd.to_numeric(data['subvented'], errors = 'coerce').fillna(3)
    return data

In [27]:
umRatePath = 'U:\\GitHubClones\\SALTSeniorDesign\\' + 'State_level_unemployment_rate.csv'
#dfForest = chargeOffs('Train', dfTapes, dealList, shelfList, umRatePath)

DRIVE 2017-1
DRIVE 2017-2
DRIVE 2017-3
SDART 2017-1
SDART 2017-2
SDART 2017-3


In [80]:
aggChargeOffsPath = 'U:\\GitHubClones\\SALTSeniorDesign\\' + 'AggregateChargeOffs.csv'
dfForest = pd.read_csv(aggChargeOffsPath, memory_map=True, low_memory=False)

In [81]:
dfForest = clean(dfForest)
MlRemoveCols = ['obligorGeographicLocation','assetNumber', 'reportingPeriodBeginningDate', 'reportingPeriodEndingDate','zeroBalanceCode']
dfForest = dfForest.drop(MlRemoveCols, axis=1)
dfForest = dfForest.astype(np.float64)
dfForest.head()

,originalLoanAmount,originalLoanTerm,originalInterestRatePercentage,subvented,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,remainingTermToMaturityNumber,reportingPeriodBeginningLoanBalanceAmount,reportingPeriodActualEndBalanceAmount,totalActualAmountPaid,currentDelinquencyStatus,LTV,loanDiff,StateUnemployment
0,15739.00,60.0,0.2100,0.0,2.0,2008.0,1.0,14250.0,0.0,3.0,3.0,1.0,0.076434,26.0,8776.39,0.0,8872.33,0.0,110.449123,6962.61,4.3
1,15665.75,72.0,0.2400,0.0,2.0,2010.0,1.0,12775.0,548.0,3.0,2.0,0.0,0.125043,13.0,4739.21,0.0,4911.36,0.0,122.628180,10926.54,3.2
2,13046.57,72.0,0.2499,0.0,2.0,2005.0,1.0,12599.0,470.0,3.0,3.0,0.0,0.143080,12.0,4143.22,0.0,4323.71,0.0,103.552425,8903.35,5.1
3,13942.00,72.0,0.2410,0.0,2.0,2005.0,2.0,11599.0,518.0,3.0,3.0,0.0,0.147815,13.0,2384.08,0.0,2431.30,0.0,120.200017,11557.92,4.6
4,13648.16,72.0,0.2499,0.0,2.0,2009.0,1.0,11998.0,0.0,3.0,3.0,1.0,0.140553,12.0,4109.14,0.0,4218.86,0.0,113.753626,9539.02,4.7


In [51]:
'''
forestCols = list(dfForest.columns)
forestCols

nonStandardizedList = ['subvented', 'zeroBalanceCode', 'obligorIncomeVerificationLevelCode',
              'obligorEmploymentVerificationCode','coObligorIndicator','vehicleTypeCode','vehicleNewUsedCode',]
for i in range(0,len(nonStandardList)):
    if nonStandardList[i] in forestCols: forestCols.remove(nonStandardList[i])
        '''

In [52]:
'''
standardizedForest = dfForest[forestCols]
nonStandardizedForest = dfForest[nonStandardizedList]
dfForest_scale = preprocessing.StandardScaler().fit_transform(standardizedForest)

standardizedForest = pd.DataFrame(dfForest_scale, columns=standardizedForest.columns)
dfForest = pd.concat([standardizedForest, nonStandardizedForest], axis=1)'''

In [82]:
dfForest_scale = pd.DataFrame(preprocessing.scale(dfForest), columns=dfForest.columns)

In [88]:
pca = PCA(n_components=15)
pca.fit_transform(dfForest_scale)
pd.DataFrame(pca.components_,columns=dfForest_scale.columns)

,originalLoanAmount,originalLoanTerm,originalInterestRatePercentage,subvented,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,remainingTermToMaturityNumber,reportingPeriodBeginningLoanBalanceAmount,reportingPeriodActualEndBalanceAmount,totalActualAmountPaid,currentDelinquencyStatus,LTV,loanDiff,StateUnemployment
0,0.374626,0.174774,-0.218775,0.151429,-0.267722,0.322710,0.146289,0.406485,0.119296,-0.151909,-0.125859,0.017073,0.035189,0.229802,0.380204,-0.000000e+00,0.324340,-0.108750,-0.137248,-0.089194,0.003356
1,0.270515,0.024978,-0.348409,0.229685,-0.116515,-0.234246,0.117416,0.199087,0.265957,0.088361,0.080870,0.077071,-0.049867,-0.436400,-0.226065,1.323489e-23,-0.095605,-0.007382,0.109173,0.523449,0.003679
2,0.095966,0.248802,0.257895,0.207908,-0.332358,0.238579,-0.130511,0.088818,-0.238584,0.198137,0.195855,-0.144941,0.346735,0.096641,-0.065824,0.000000e+00,-0.314609,0.456136,-0.040616,0.168816,-0.014719
3,0.104914,0.069875,0.110510,-0.041410,0.118357,-0.107533,0.090185,0.045779,0.022790,0.604515,0.607690,0.134771,0.033164,-0.034731,0.222266,-0.000000e+00,0.259492,-0.150653,0.114650,-0.160949,-0.013554
4,-0.189614,-0.399688,-0.054167,0.198324,-0.221586,0.014889,0.119388,0.141829,-0.345291,0.109263,0.136653,-0.088227,-0.123839,-0.162456,-0.119414,-0.000000e+00,0.029055,-0.191371,-0.652307,-0.040605,-0.010396
5,-0.081439,0.110487,0.000769,0.078448,-0.165489,0.208300,-0.156497,-0.048417,0.030370,0.073364,0.045760,0.675089,-0.583715,0.152633,-0.070104,0.000000e+00,-0.143748,0.129019,-0.041700,0.004656,-0.072423
6,0.085002,-0.180268,0.094851,-0.078912,0.125012,-0.102278,0.520822,0.078595,-0.052208,-0.056859,-0.063360,0.167153,0.074268,0.017167,0.093139,-5.551115e-17,-0.106695,0.285673,-0.007288,-0.028308,-0.703766
7,0.131360,-0.113120,0.205227,-0.055497,0.105402,-0.132233,0.470824,0.108944,-0.181159,-0.045525,-0.095586,0.218444,-0.057379,-0.030845,0.108166,0.000000e+00,-0.068211,0.280629,0.009521,-0.001743,0.686603
8,-0.109691,-0.176485,0.055854,0.793415,-0.121371,-0.092536,0.060844,-0.262534,0.008872,-0.043708,-0.090684,0.064283,0.080052,-0.063725,0.137706,-0.000000e+00,0.054197,-0.024434,0.331716,-0.266316,-0.001316
9,-0.091151,-0.397242,-0.251837,-0.092183,-0.099860,0.142024,0.006156,-0.037392,0.584019,0.117954,0.162551,-0.226686,-0.069755,0.062448,0.127881,-8.326673e-17,-0.224480,0.383689,-0.067149,-0.237098,0.118908
